In [2]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [3]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [4]:
# init
steepness = 50    # steepness factor
num_blocks = 10   # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [5]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [6]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 32
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [7]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))[:100]):
    # set params
    model.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
df.to_csv("result/rb_exact_50-10.csv")
df = df.iloc[:100]
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:40:13<00:00, 60.14s/it]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    56.299994               0.0     60.135377
std     29.319991               0.0      0.337887
min     14.076352               0.0     59.336500
25%     36.253682               0.0     59.941961
50%     54.563601               0.0     60.075549
75%     71.377076               0.0     60.252838
max    193.248342               0.0     61.158490
Number of infeasible solution: 0


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_heur_rnd_50-10.csv")
df = df.iloc[:100]
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [20:05<00:00,  1.21s/it]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    49.669134               0.0      1.146176
std     15.012969               0.0      0.269541
min     24.588444               0.0      0.799154
25%     39.523352               0.0      0.987704
50%     46.571316               0.0      1.124284
75%     59.712077               0.0      1.241595
max     95.044551               0.0      2.107445
Number of infeasible solution: 0


In [8]:
df = pd.read_csv("result/rb_heur_rnd_50-10.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)+df["Sol"].isna().sum()))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    49.669134               0.0      1.146176
std     15.012969               0.0      0.269541
min     24.588444               0.0      0.799154
25%     39.523352               0.0      0.987704
50%     46.571316               0.0      1.124284
75%     59.712077               0.0      1.241595
max     95.044551               0.0      2.107445
Number of infeasible solution: 0


## Heuristic - N1

In [7]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n1_50-10.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:48<00:00,  5.92it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean     86.607460               0.0      0.167827
std      33.153481               0.0      0.056185
min      18.894174               0.0      0.092560
25%      61.266989               0.0      0.135658
50%      80.064215               0.0      0.153602
75%     108.016210               0.0      0.186170
max     197.637394               0.0      0.763410
Number of infeasible solution: 0


In [10]:
df = pd.read_csv("result/rb_heur_n1_50-10.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    88.140343               0.0      0.189450
std     34.004166               0.0      0.078168
min     32.800968               0.0      0.121152
25%     62.629849               0.0      0.141907
50%     79.042608               0.0      0.170548
75%    109.904937               0.0      0.207376
max    193.248342               0.0      0.763410
Number of infeasible solution: 0


## Learnable Rounding

In [9]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [10]:
# hyperparameters
penalty_weight = 60   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [11]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [12]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 1704.93
Epoch 1, Validation Loss: 387.71
Epoch 2, Validation Loss: 192.06
Epoch 3, Validation Loss: 154.03
Epoch 4, Validation Loss: 119.63
Epoch 5, Validation Loss: 100.79
Epoch 6, Validation Loss: 97.23
Epoch 7, Validation Loss: 90.80
Epoch 8, Validation Loss: 73.20
Epoch 9, Validation Loss: 80.31
Epoch 10, Validation Loss: 77.94
Epoch 11, Validation Loss: 68.36
Epoch 12, Validation Loss: 77.64
Epoch 13, Validation Loss: 74.17
Epoch 14, Validation Loss: 67.03
Epoch 15, Validation Loss: 65.92
Epoch 16, Validation Loss: 65.04
Epoch 17, Validation Loss: 58.32
Epoch 18, Validation Loss: 60.63
Epoch 19, Validation Loss: 64.26
Epoch 20, Validation Loss: 74.40
Epoch 21, Validation Loss: 75.13
Epoch 22, Validation Loss: 68.91
Epoch 23, Validation Loss: 64.21
Epoch 24, Validation Loss: 58.74
Epoch 25, Validation Loss: 60.76
Epoch 26, Validation Loss: 64.89
Epoch 27, Validation Loss: 54.73
Epoch 28, Validation Loss: 61.64
Epoch 29, Validation Loss: 61.32
Epoch 30, Val

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lr_50-10.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 200.98it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     48.822808          0.002931      0.002664
std      31.285391          0.085363      0.000712
min       5.257703          0.000000      0.001000
25%      25.840578          0.000000      0.002002
50%      40.305008          0.000000      0.002516
75%      64.697848          0.000000      0.003002
max     217.889724          2.692807      0.006518
Number of infeasible solution: 3


In [11]:
df = pd.read_csv("result/rb_lr_50-10.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    48.218733               0.0      0.002845
std     37.224549               0.0      0.000796
min     10.719886               0.0      0.001000
25%     23.261847               0.0      0.002005
50%     37.672530               0.0      0.003000
75%     59.583418               0.0      0.003009
max    217.889724               0.0      0.006518
Number of infeasible solution: 0


## Learnable Threshold

In [14]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [15]:
# hyperparameters
penalty_weight = 60   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [16]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [17]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 1410.69
Epoch 1, Validation Loss: 235.52
Epoch 2, Validation Loss: 106.07
Epoch 3, Validation Loss: 85.14
Epoch 4, Validation Loss: 94.56
Epoch 5, Validation Loss: 83.00
Epoch 6, Validation Loss: 79.81
Epoch 7, Validation Loss: 78.99
Epoch 8, Validation Loss: 78.03
Epoch 9, Validation Loss: 81.62
Epoch 10, Validation Loss: 69.11
Epoch 11, Validation Loss: 70.51
Epoch 12, Validation Loss: 72.32
Epoch 13, Validation Loss: 69.13
Epoch 14, Validation Loss: 69.14
Epoch 15, Validation Loss: 67.26
Epoch 16, Validation Loss: 73.56
Epoch 17, Validation Loss: 71.02
Epoch 18, Validation Loss: 69.29
Epoch 19, Validation Loss: 67.08
Epoch 20, Validation Loss: 66.12
Epoch 21, Validation Loss: 71.69
Epoch 22, Validation Loss: 65.39
Epoch 23, Validation Loss: 68.40
Epoch 24, Validation Loss: 67.37
Epoch 25, Validation Loss: 72.51
Epoch 26, Validation Loss: 66.58
Epoch 27, Validation Loss: 74.93
Epoch 28, Validation Loss: 67.20
Epoch 29, Validation Loss: 68.08
Epoch 30, Valida

In [18]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lt_50-10.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 208.27it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     60.428413          0.007411      0.002564
std      28.787232          0.090534      0.000733
min      12.746076          0.000000      0.000996
25%      36.438821          0.000000      0.002000
50%      52.429313          0.000000      0.002506
75%      84.480311          0.000000      0.003000
max     166.475645          2.048779      0.008211
Number of infeasible solution: 12


In [12]:
df = pd.read_csv("result/rb_lt_50-10.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    63.718770               0.0      0.002942
std     32.265123               0.0      0.001112
min     20.026440               0.0      0.001000
25%     35.759567               0.0      0.002000
50%     59.211945               0.0      0.003000
75%     87.684815               0.0      0.003511
max    143.387911               0.0      0.008211
Number of infeasible solution: 0


## Parametric Learning Then Rounding

In [13]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [14]:
# hyperparameters
penalty_weight = 60   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [16]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x"], steepness, num_blocks, penalty_weight)

In [17]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 1401.97
Epoch 1, Validation Loss: 125.00
Epoch 2, Validation Loss: 107.21
Epoch 3, Validation Loss: 65.04
Epoch 4, Validation Loss: 38.25
Epoch 5, Validation Loss: 34.85
Epoch 6, Validation Loss: 29.72
Epoch 7, Validation Loss: 28.84
Epoch 8, Validation Loss: 28.40
Epoch 9, Validation Loss: 28.32
Epoch 10, Validation Loss: 34.94
Epoch 11, Validation Loss: 26.64
Epoch 12, Validation Loss: 27.60
Epoch 13, Validation Loss: 26.34
Epoch 14, Validation Loss: 23.90
Epoch 15, Validation Loss: 24.27
Epoch 16, Validation Loss: 23.18
Epoch 17, Validation Loss: 23.20
Epoch 18, Validation Loss: 21.43
Epoch 19, Validation Loss: 21.39
Epoch 20, Validation Loss: 21.60
Epoch 21, Validation Loss: 24.08
Epoch 22, Validation Loss: 21.63
Epoch 23, Validation Loss: 21.97
Epoch 24, Validation Loss: 21.31
Epoch 25, Validation Loss: 24.17
Epoch 26, Validation Loss: 19.84
Epoch 27, Validation Loss: 20.74
Epoch 28, Validation Loss: 23.39
Epoch 29, Validation Loss: 23.10
Epoch 30, Valida

In [18]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x"]
    for i in range(num_blocks*2):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_pr_50-10.csv")
df = df.iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 333.94it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    60.192886               0.0      0.000492
std     28.203855               0.0      0.000722
min     14.495794               0.0      0.000000
25%     38.724998               0.0      0.000000
50%     57.974880               0.0      0.000000
75%     76.232206               0.0      0.001000
max    142.242361               0.0      0.005659
Number of infeasible solution: 0
